In [2]:
from matplotlib import pyplot
import pandas as pd
import numpy as np
from datetime import datetime
from geopy.distance import vincenty
from geopy.distance import great_circle
import math

In [3]:
train=pd.read_csv('/home/harsh/dataset/train.csv')
train_fare=train['fare_amount']

# test=pd.read_csv('test.csv')

In [4]:
[nr,nc]=train.shape

# temp=train['tip_amount'].mode()
# print 'harsh'
# print temp
# print 'kara'
# # train['tip_amount'].fillna(0.0,inplace=True)
# # print train.isnull().sum()
# train.dtypes



In [ ]:
# a=pd.DataFrame(np.ones((3,1)))
# a['cv']=[1,3,2]
# a['klp']=a['poi']=np.ones(3)
# a

In [5]:
fmt='%Y-%m-%d %H:%M:%S'

train['pickup_datetime']=pd.to_datetime(train['pickup_datetime'])
train['dropoff_datetime']=pd.to_datetime(train['dropoff_datetime'])

train['tot_time']=(((train['dropoff_datetime']-train['pickup_datetime']).astype('timedelta64[s]'))/60).round()
# train['day']= train['pickup_datetime'].dt.day
# train['month'] = train['pickup_datetime'].dt.month
train['year'] = train['pickup_datetime'].dt.year

train['weekday'] = train['pickup_datetime'].dt.dayofweek   

train['hour'] = train['pickup_datetime'].dt.hour
    

In [6]:
#dropping the instances where the time is -ve
a=(train['tot_time']<0)
ind=train['tot_time'][a].index
print len(ind)
print train.shape
train.drop(ind,axis=0,inplace=True)
print train.shape


35
(1658099, 22)
(1658064, 22)


In [ ]:
# train.loc[train.index[[0,1,2,3,4,5]]]
# train[['tip_amount']]

In [7]:
#train.index is used because in case we drop off some of the rows from dataframe, there can still be accessed using the index
#suppose we delete the row 2,3,4
#then the index of train will have 0,1,5,6,7,...
#but if we access the 2nd row  using the train.loc([2]), then the removed row will be accessed.
# so we should first find the indices which have the data
# however if we use the train.iloc([[2]]) then the 2nd valid row will be accessed.
train['tot_dist'] = [ great_circle( (train['pickup_latitude'][i],train['pickup_longitude'][i]) , (train['dropoff_latitude'][i],train['dropoff_longitude'][i])  ).km   for i in train.index ]

In [ ]:
#even if the dist=0 and fare is not zero can be a true case, in case of ROUND TRIP
#(GO SOMEWHERE AND COMES TO SAME PLACE IN SAME CAB)

# a=(train['tot_dist']==0)
# ind1=train[a].index

# a=(train['fare_amount']==0)
# ind2=train[a].index

# train.loc[ind1]

#see whether to drop these rows or not
# train.drop(ind,axis=0,inplace=True)

In [8]:
categorical_cols=['payment_type','vendor_id','new_user']
new_feat = pd.get_dummies(train[categorical_cols])
print new_feat.columns


Index([u'payment_type_CRD', u'payment_type_CSH', u'payment_type_DIS',
       u'payment_type_NOC', u'payment_type_UNK', u'vendor_id_DST000401',
       u'vendor_id_DST000481', u'vendor_id_DST000532', u'vendor_id_DST000543',
       u'new_user_NO', u'new_user_YES'],
      dtype='object')


In [9]:
useless_cols=['TID','vendor_id','new_user','pickup_datetime','dropoff_datetime','dropoff_longitude','dropoff_latitude','payment_type','store_and_fwd_flag']
train.drop(useless_cols,axis=1,inplace=True)
train=pd.concat([train,new_feat],axis=1)
train.dtypes

tolls_amount           float64
tip_amount             float64
mta_tax                float64
passenger_count          int64
pickup_longitude       float64
pickup_latitude        float64
rate_code                int64
surcharge              float64
fare_amount            float64
tot_time               float64
year                     int64
weekday                  int64
hour                     int64
tot_dist               float64
payment_type_CRD         uint8
payment_type_CSH         uint8
payment_type_DIS         uint8
payment_type_NOC         uint8
payment_type_UNK         uint8
vendor_id_DST000401      uint8
vendor_id_DST000481      uint8
vendor_id_DST000532      uint8
vendor_id_DST000543      uint8
new_user_NO              uint8
new_user_YES             uint8
dtype: object

In [16]:
train.isnull().sum()

tolls_amount                0
tip_amount             165192
mta_tax                     0
passenger_count             0
pickup_longitude        49809
pickup_latitude         33099
rate_code                   0
surcharge               99172
fare_amount                 0
tot_time                    0
year                        0
weekday                     0
hour                        0
tot_dist                94366
payment_type_CRD            0
payment_type_CSH            0
payment_type_DIS            0
payment_type_NOC            0
payment_type_UNK            0
vendor_id_DST000401         0
vendor_id_DST000481         0
vendor_id_DST000532         0
vendor_id_DST000543         0
new_user_NO                 0
new_user_YES                0
dtype: int64

In [17]:
#filling the missing values
m=train['tip_amount'].mode()[0]
train['tip_amount'].fillna(m,inplace=True)

m=train['surcharge'].mode()[0]
train['surcharge'].fillna(m,inplace=True)

m=train['pickup_latitude'].mode()[0]
train['pickup_latitude'].fillna(m,inplace=True)

m=train['pickup_longitude'].mode()[0]
train['pickup_longitude'].fillna(m,inplace=True)

m=train['tot_dist'].mode()[0]
train['tot_dist'].fillna(m,inplace=True)

train.isnull().sum()

# a=pd.Series([2,1,np.nan,5,2,1,1,1,4])
# print m.mode()[0]

tolls_amount           0
tip_amount             0
mta_tax                0
passenger_count        0
pickup_longitude       0
pickup_latitude        0
rate_code              0
surcharge              0
fare_amount            0
tot_time               0
year                   0
weekday                0
hour                   0
tot_dist               0
payment_type_CRD       0
payment_type_CSH       0
payment_type_DIS       0
payment_type_NOC       0
payment_type_UNK       0
vendor_id_DST000401    0
vendor_id_DST000481    0
vendor_id_DST000532    0
vendor_id_DST000543    0
new_user_NO            0
new_user_YES           0
dtype: int64

In [31]:
a=[1,2,3,4,5]
b=[2,4]
c=set(a)-set(b)
print list(c)
train.loc[list(c)]

[1, 3, 5]


,tolls_amount,tip_amount,mta_tax,passenger_count,pickup_longitude,pickup_latitude,rate_code,surcharge,fare_amount,tot_time,...,payment_type_CSH,payment_type_DIS,payment_type_NOC,payment_type_UNK,vendor_id_DST000401,vendor_id_DST000481,vendor_id_DST000532,vendor_id_DST000543,new_user_NO,new_user_YES
1,0.0,1.0,0.5,3,-73.973864,40.752194,1,0.0,8.5,6.0,...,0,0,0,0,1,0,0,0,1,0
3,0.0,1.8,0.5,2,-73.962345,40.767215,1,0.0,11.3,12.0,...,0,0,0,0,0,0,1,0,1,0
5,0.0,2.4,0.5,1,-73.995571,40.716019,1,0.0,14.4,12.0,...,0,0,0,0,1,0,0,0,1,0


In [32]:
a=(train['year']<=2015)
ind=train[a].index

train_2015=train.loc[ind]
print train_2015.shape

train_2016=train.loc[set(train.index)-set(ind)]
print train_2016.shape

(1142170, 25)
(515894, 25)


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
X = train_2016[list(set(train_2016.columns)-set(['fare_amount']))]
Y = train_2016['fare_amount']
num_folds = 10
kfold = KFold(n_splits=10, random_state=7)
model = DecisionTreeRegressor()
scoring = 'neg_mean_absolute_error'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print 'cross-val-error is'
print(results.mean())


In [39]:
X_2015= train_2015[list(set(train_2015.columns)-set(['fare_amount']))]
Y_2015 = train_2015['fare_amount']
model=DecisionTreeRegressor()
model.fit(X_2015,Y_2015)

print 'training over'
X_2016= train_2016[list(set(train_2016.columns)-set(['fare_amount']))]
Y_2016 = train_2016['fare_amount']
predict = model.predict(X_2016)

((predict-Y_2016).abs().sum())/X_2016.shape[0]

training over


1.41356833675677